# COCO to Masks Conversion - CORRECTED VERSION

## ⚠️ Important: Polygon-Based Segmentation

This notebook converts COCO format annotations to binary segmentation masks.

**Key Fix Applied:**
- ✅ **Using `segmentation` polygons** - Extracts actual object shapes
- ❌ ~~Using `bbox` rectangles~~ - Would create rectangular masks (WRONG!)

## Process:
1. **Generate Cracks Masks** - from `cracks.v1i.coco` dataset
2. **Generate Drywall Masks** - from `Drywall-Join-Detect.v1i.coco` dataset  
3. **Merge Datasets** - Copy images and masks to `final_dataset/`

## Output Format:
- **Mask files**: `{image_id}__segment_crack.png` or `{image_id}__segment_taping_area.png`
- **Values**: 0 (background) and 255 (segmented region)
- **Format**: Single-channel PNG

## ✅ CORRECTED: Generate Masks with Full Roboflow Filenames

This version preserves the FULL Roboflow filename (including `.rf.XXXXX`) to ensure exact matching between images and masks.

**Why this matters:**
- Each base image (e.g., `00002_jpg`) has multiple augmented versions (`00002_jpg.rf.abc123`, `00002_jpg.rf.def456`, etc.)
- Each augmented version has different annotations
- We must keep the full filename to match the correct image with its corresponding mask

**Bbox Fallback (CRITICAL FIX):**
- Some datasets (e.g., Drywall) were exported as object detection format with empty `segmentation` arrays
- When `segmentation` is empty, we use the `bbox` (bounding box) as a fallback to create rectangular masks
- This ensures all samples have valid ground truth for training

In [ ]:
import os
import json
import cv2
import numpy as np

def generate_masks_with_full_filenames(dataset_root, output_root, prompt_name):
    """Generate masks keeping the FULL Roboflow filename to match images exactly"""
    
    splits = ["train", "valid", "test"]
    
    for split in splits:
        split_path = os.path.join(dataset_root, split)
        json_path = os.path.join(split_path, "_annotations.coco.json")
        
        if not os.path.exists(json_path):
            print(f"Skipping {split} (no annotations)")
            continue
        
        out_dir = os.path.join(output_root, split)
        os.makedirs(out_dir, exist_ok=True)
        
        with open(json_path) as f:
            coco = json.load(f)
        
        images = {img["id"]: img for img in coco["images"]}
        
        ann_by_image = {}
        for ann in coco["annotations"]:
            ann_by_image.setdefault(ann["image_id"], []).append(ann)
        
        print(f"\nProcessing {split}...")
        count = 0
        for image_id, img_info in images.items():
            count += 1
            if count % 500 == 0:
                print(f"  Processed {count}/{len(images)} images")
                
            width = img_info["width"]
            height = img_info["height"]
            filename = img_info["file_name"]  # e.g., "00002_jpg.rf.xxx.jpg"
            
            # Create empty mask
            mask = np.zeros((height, width), dtype=np.uint8)
            
            anns = ann_by_image.get(image_id, [])
            
            # Draw each segmentation polygon (or bbox fallback)
            for ann in anns:
                if 'segmentation' in ann and ann['segmentation']:
                    # Use polygon segmentation
                    for seg in ann['segmentation']:
                        poly = np.array(seg).reshape(-1, 2).astype(np.int32)
                        cv2.fillPoly(mask, [poly], 255)
                elif 'bbox' in ann:
                    # Fallback: use bounding box when segmentation is empty
                    x, y, w, h = ann['bbox']
                    x1, y1 = int(x), int(y)
                    x2, y2 = int(x + w), int(y + h)
                    cv2.rectangle(mask, (x1, y1), (x2, y2), 255, -1)  # -1 fills the rectangle
            
            # ✅ KEY FIX: Keep the FULL filename (not just base)
            # Remove extension, add prompt
            base_name = os.path.splitext(filename)[0]  # "00002_jpg.rf.xxx"
            out_name = base_name + "__" + prompt_name + ".png"
            cv2.imwrite(os.path.join(out_dir, out_name), mask)
        
        print(f"  Completed {count} images for {split}")
    
    print(f"✅ Done with {dataset_root}!")

# print("Function defined with bbox fallback!")

Function defined with bbox fallback!


In [2]:
# Generate masks for cracks dataset
generate_masks_with_full_filenames(
    dataset_root="cracks.v1i.coco",
    output_root="generated_masks/cracks",
    prompt_name="segment_crack"
)

# Generate masks for drywall dataset  
generate_masks_with_full_filenames(
    dataset_root="Drywall-Join-Detect.v1i.coco",
    output_root="generated_masks/drywall",
    prompt_name="segment_taping_area"
)

print("\n✅ All masks generated with full filenames!")


Processing train...
  Processed 500/5164 images
  Processed 500/5164 images
  Processed 1000/5164 images
  Processed 1000/5164 images
  Processed 1500/5164 images
  Processed 1500/5164 images
  Processed 2000/5164 images
  Processed 2000/5164 images
  Processed 2500/5164 images
  Processed 2500/5164 images
  Processed 3000/5164 images
  Processed 3000/5164 images
  Processed 3500/5164 images
  Processed 3500/5164 images
  Processed 4000/5164 images
  Processed 4000/5164 images
  Processed 4500/5164 images
  Processed 4500/5164 images
  Processed 5000/5164 images
  Processed 5000/5164 images
  Completed 5164 images for train

Processing valid...
  Completed 5164 images for train

Processing valid...
  Completed 201 images for valid

Processing test...
  Completed 4 images for test
✅ Done with cracks.v1i.coco!

Processing train...
  Completed 201 images for valid

Processing test...
  Completed 4 images for test
✅ Done with cracks.v1i.coco!

Processing train...
  Processed 500/820 image

## Merge Corrected Datasets with Exact Matching

Copy images and masks to `final_dataset/` with exact filename matching (including Roboflow augmentation IDs).

In [3]:
import shutil

splits = ['train', 'valid']

for split in splits:
    img_out = os.path.join("final_dataset", split, "images")
    mask_out = os.path.join("final_dataset", split, "masks")
    
    # Clear old data
    if os.path.exists(img_out):
        for f in os.listdir(img_out):
            os.remove(os.path.join(img_out, f))
    if os.path.exists(mask_out):
        for f in os.listdir(mask_out):
            os.remove(os.path.join(mask_out, f))
    
    os.makedirs(img_out, exist_ok=True)
    os.makedirs(mask_out, exist_ok=True)
    
    # Process cracks
    cracks_img_dir = os.path.join("cracks.v1i.coco", split)
    cracks_mask_dir = os.path.join("generated_masks/cracks", split)
    
    if os.path.exists(cracks_mask_dir):
        print(f"\nCopying cracks {split}...")
        count = 0
        for mask_fname in os.listdir(cracks_mask_dir):
            # mask_fname is like: "00002_jpg.rf.xxx__segment_crack.png"
            base_with_rf = mask_fname.split("__")[0]  # "00002_jpg.rf.xxx"
            
            # Find matching image
            img_path = os.path.join(cracks_img_dir, base_with_rf + ".jpg")
            if os.path.exists(img_path):
                # Copy image with full Roboflow filename
                shutil.copy(img_path, os.path.join(img_out, base_with_rf + ".jpg"))
                # Copy mask
                shutil.copy(os.path.join(cracks_mask_dir, mask_fname), os.path.join(mask_out, mask_fname))
                count += 1
        print(f"  Copied {count} crack samples")
    
    # Process drywall
    drywall_img_dir = os.path.join("Drywall-Join-Detect.v1i.coco", split)
    drywall_mask_dir = os.path.join("generated_masks/drywall", split)
    
    if os.path.exists(drywall_mask_dir):
        print(f"\nCopying drywall {split}...")
        count = 0
        for mask_fname in os.listdir(drywall_mask_dir):
            base_with_rf = mask_fname.split("__")[0]
            
            img_path = os.path.join(drywall_img_dir, base_with_rf + ".jpg")
            if os.path.exists(img_path):
                shutil.copy(img_path, os.path.join(img_out, base_with_rf + ".jpg"))
                shutil.copy(os.path.join(drywall_mask_dir, mask_fname), os.path.join(mask_out, mask_fname))
                count += 1
        print(f"  Copied {count} drywall samples")

# Verify final counts
train_imgs = len(os.listdir("final_dataset/train/images"))
train_masks = len(os.listdir("final_dataset/train/masks"))
valid_imgs = len(os.listdir("final_dataset/valid/images"))
valid_masks = len(os.listdir("final_dataset/valid/masks"))

print("\n" + "="*70)
print("✅ FINAL DATASET READY!")
print("="*70)
print(f"Train: {train_imgs} images, {train_masks} masks")
print(f"Valid: {valid_imgs} images, {valid_masks} masks")

if train_imgs == train_masks and valid_imgs == valid_masks:
    print("✅ Image-mask counts match perfectly!")
else:
    print("⚠️ Warning: Image-mask count mismatch!")


Copying cracks train...
  Copied 5164 crack samples

Copying drywall train...
  Copied 820 drywall samples

Copying cracks valid...
  Copied 201 crack samples

Copying drywall valid...
  Copied 202 drywall samples

✅ FINAL DATASET READY!
Train: 5984 images, 5984 masks
Valid: 403 images, 403 masks
✅ Image-mask counts match perfectly!
  Copied 5164 crack samples

Copying drywall train...
  Copied 820 drywall samples

Copying cracks valid...
  Copied 201 crack samples

Copying drywall valid...
  Copied 202 drywall samples

✅ FINAL DATASET READY!
Train: 5984 images, 5984 masks
Valid: 403 images, 403 masks
✅ Image-mask counts match perfectly!
